**Step 1**: Install Spark

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# New Section

In [ ]:
# innstall java
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

# install spark (change the version number if needed)
!wget -q https://archive.apache.org/dist/spark/spark-3.0.0/spark-3.0.0-bin-hadoop3.2.tgz

# unzip the spark file to the current folder
!tar xf spark-3.0.0-bin-hadoop3.2.tgz

# New Section

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.0-bin-hadoop3.2"

In [ ]:
# install findspark using pip
!pip install -q findspark

In [ ]:
!pip3 install pyspark==3.2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.3/281.3 MB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.8/198.8 kB 26.4 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.2.0-py2.py3-none-any.whl size=281805900 sha256=982910b83dff1270cc6a9d977f510268bff25ebb5aa015d21e6bbfe7e0f2a9a1
  Stored in directory: /root/.cache/pip/wheels/32/97/d3/8b6d964c8700e4fbb561c71638a92ec55dac9be51eb5fea86d
Successfully built pyspark
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.7
    Uninstalling py4j-0.10.9.7:
      Successfully uninstalled py4j-0.10.9.7


In [ ]:
import findspark
findspark.init()

In [ ]:
# Import SparkSession
from pyspark.sql import SparkSession
# Create a Spark Session
spark = SparkSession.builder.master("local[*]").getOrCreate()
# Check Spark Session Information
spark

In [ ]:
# Create spark_session
spark_session = SparkSession.builder.getOrCreate()

In [ ]:
sc = spark_session.sparkContext

In [ ]:
logfilesrdd = sc.textFile("/content/drive/My Drive/Colab Notebooks/datasets/housing.txt")

In [ ]:
print(type(logfilesrdd))

<class 'pyspark.rdd.RDD'>


In [ ]:
print(logfilesrdd.count())

506


**Setting up Spark**:

Before you can connect to a Spark cluster, Spark needs to be installed. The code below is boilerplate code that can be used to set-up Spark. Please note that this code will be leveraged in all the notebooks since each nodebook is a separate entity.

List the contents of Google Drive

In [ ]:
# After executing the cell above, Drive
# files will be present in "/content/My Drive".
!ls "/content/drive/My Drive"

'2022冬季1班 LaiData 第一节课 试听课 12月05日 7PM (美西时间).gdoc'
'Authorization Code.docx'
'Colab Notebooks'
 Documents
 Others
 zhangyu027_token_2023.rtf


**Using DataFrames**

Spark's main data structure is the Resilient Distributed Dataset (RDD) which is a low level object that allows Spark to process large quantities of data by spreading data across multiple nodes in the cluster. However, RDDs are hard to work with directly; therefore, park DataFrame will be utilized since it provides an abstraction built on top of RDDs.

The Spark DataFrame behaves a lot like a SQL table (a table contains attributes in the columns and observations in the rows). Not only are DataFrames easier to work with, they are also more optimized for complicated operations than RDDs.

Read the Airlines.csv file from your Google Drive into a Spark DataFrame

In [ ]:
df = spark_session.read.option("header", "true").csv("/content/drive/My Drive/Colab Notebooks/datasets/airline.csv")

Display the contents of the DataFrame

In [ ]:
df.show(100)

+----+-----+----------+---------+-------+----------+-------+----------+-------------+---------+-------+-----------------+--------------+-------+--------+--------+------+----+--------+------+-------+---------+----------------+--------+------------+------------+--------+-------------+-----------------+
|Year|Month|DayofMonth|DayOfWeek|DepTime|CRSDepTime|ArrTime|CRSArrTime|UniqueCarrier|FlightNum|TailNum|ActualElapsedTime|CRSElapsedTime|AirTime|ArrDelay|DepDelay|Origin|Dest|Distance|TaxiIn|TaxiOut|Cancelled|CancellationCode|Diverted|CarrierDelay|WeatherDelay|NASDelay|SecurityDelay|LateAircraftDelay|
+----+-----+----------+---------+-------+----------+-------+----------+-------------+---------+-------+-----------------+--------------+-------+--------+--------+------+----+--------+------+-------+---------+----------------+--------+------------+------------+--------+-------------+-----------------+
|2005|    1|         1|        6|   1211|      1216|   1451|      1502|           UA|      548

Output the column names and the "rows" and "column" counts

In [ ]:
#outputing the column names
df.columns #Column Names


['Year',
 'Month',
 'DayofMonth',
 'DayOfWeek',
 'DepTime',
 'CRSDepTime',
 'ArrTime',
 'CRSArrTime',
 'UniqueCarrier',
 'FlightNum',
 'TailNum',
 'ActualElapsedTime',
 'CRSElapsedTime',
 'AirTime',
 'ArrDelay',
 'DepDelay',
 'Origin',
 'Dest',
 'Distance',
 'TaxiIn',
 'TaxiOut',
 'Cancelled',
 'CancellationCode',
 'Diverted',
 'CarrierDelay',
 'WeatherDelay',
 'NASDelay',
 'SecurityDelay',
 'LateAircraftDelay']

In [ ]:
#Outputing the row count
df.count()  #Row Count

539895

In [ ]:
#Output the column count
len(df.columns) #Column Count

29

In order to examine the summary of any particular column of a DataFrame, we use the describe method. 

The describe "method" gives us the statistical summary of the given column

In [ ]:
#Examine the summary of the "DepDelay" column
df.describe('DepDelay').show()

+-------+------------------+
|summary|          DepDelay|
+-------+------------------+
|  count|            539895|
|   mean| 11.16137993946179|
| stddev|34.378815701851316|
|    min|                -1|
|    max|                NA|
+-------+------------------+



In order to select particular columns from the DataFrame, use the select method

In [ ]:
df.select('ArrDelay','DepDelay').show()

+--------+--------+
|ArrDelay|DepDelay|
+--------+--------+
|     -11|      -5|
|     -15|      -7|
|      -8|      -3|
|      NA|      NA|
|       2|      -5|
|       1|      -1|
|      75|      75|
|     -17|      -2|
|     -22|      -9|
|      65|      83|
|     -16|      -4|
|     -22|      -7|
|     -23|      -5|
|     -14|       3|
|     -17|      -6|
|     -16|      -4|
|     -15|      -6|
|     -13|      -4|
|     -10|      -5|
|      -1|       2|
+--------+--------+
only showing top 20 rows



Selecting Distinct Multiple Columns

In [ ]:
df.select('ArrDelay','DepDelay').distinct().show()

+--------+--------+
|ArrDelay|DepDelay|
+--------+--------+
|     -23|      -5|
|     -20|      -6|
|       5|      10|
|      58|      62|
|      41|      31|
|      26|       0|
|      71|      77|
|      37|      45|
|       6|      -9|
|       2|      22|
|      20|      29|
|      65|      73|
|     102|      83|
|      56|      33|
|     123|     140|
|      80|      33|
|      73|      68|
|      72|      66|
|      55|      41|
|      69|      77|
+--------+--------+
only showing top 20 rows



Filtering Data

In [ ]:
df.filter(df.Origin=='LGA').show()

+----+-----+----------+---------+-------+----------+-------+----------+-------------+---------+-------+-----------------+--------------+-------+--------+--------+------+----+--------+------+-------+---------+----------------+--------+------------+------------+--------+-------------+-----------------+
|Year|Month|DayofMonth|DayOfWeek|DepTime|CRSDepTime|ArrTime|CRSArrTime|UniqueCarrier|FlightNum|TailNum|ActualElapsedTime|CRSElapsedTime|AirTime|ArrDelay|DepDelay|Origin|Dest|Distance|TaxiIn|TaxiOut|Cancelled|CancellationCode|Diverted|CarrierDelay|WeatherDelay|NASDelay|SecurityDelay|LateAircraftDelay|
+----+-----+----------+---------+-------+----------+-------+----------+-------------+---------+-------+-----------------+--------------+-------+--------+--------+------+----+--------+------+-------+---------+----------------+--------+------------+------------+--------+-------------+-----------------+
+----+-----+----------+---------+-------+----------+-------+----------+-------------+---------

Filtering Data (Multiple Parameters)

In [ ]:
df.filter((df.Dest=='GSO') & (df.Origin=='LGA')).show()

+----+-----+----------+---------+-------+----------+-------+----------+-------------+---------+-------+-----------------+--------------+-------+--------+--------+------+----+--------+------+-------+---------+----------------+--------+------------+------------+--------+-------------+-----------------+
|Year|Month|DayofMonth|DayOfWeek|DepTime|CRSDepTime|ArrTime|CRSArrTime|UniqueCarrier|FlightNum|TailNum|ActualElapsedTime|CRSElapsedTime|AirTime|ArrDelay|DepDelay|Origin|Dest|Distance|TaxiIn|TaxiOut|Cancelled|CancellationCode|Diverted|CarrierDelay|WeatherDelay|NASDelay|SecurityDelay|LateAircraftDelay|
+----+-----+----------+---------+-------+----------+-------+----------+-------------+---------+-------+-----------------+--------------+-------+--------+--------+------+----+--------+------+-------+---------+----------------+--------+------------+------------+--------+-------------+-----------------+
+----+-----+----------+---------+-------+----------+-------+----------+-------------+---------

**Performing SQL Queries** - SQL queries can be passes directly to any DataFrame; for that, we need to create a table from the DataFrame using the registerTempTable

In [ ]:
df.registerTempTable('airlines')

Typically the entry point into all SQL functionality in Spark is the SQLContext class. To create a basic instance of this call, all we need is a SparkContext reference.

In [ ]:
from pyspark.sql import SQLContext
sqlContext = SQLContext(spark_session)
sqlContext

In [ ]:
sqlContext.sql('select * from airlines').show()

+----+-----+----------+---------+-------+----------+-------+----------+-------------+---------+-------+-----------------+--------------+-------+--------+--------+------+----+--------+------+-------+---------+----------------+--------+------------+------------+--------+-------------+-----------------+
|Year|Month|DayofMonth|DayOfWeek|DepTime|CRSDepTime|ArrTime|CRSArrTime|UniqueCarrier|FlightNum|TailNum|ActualElapsedTime|CRSElapsedTime|AirTime|ArrDelay|DepDelay|Origin|Dest|Distance|TaxiIn|TaxiOut|Cancelled|CancellationCode|Diverted|CarrierDelay|WeatherDelay|NASDelay|SecurityDelay|LateAircraftDelay|
+----+-----+----------+---------+-------+----------+-------+----------+-------------+---------+-------+-----------------+--------------+-------+--------+--------+------+----+--------+------+-------+---------+----------------+--------+------------+------------+--------+-------------+-----------------+
|2005|    1|         1|        6|   1211|      1216|   1451|      1502|           UA|      548

In [ ]:
sqlContext.sql('select distinct(Dest) from airlines').show(20)

+----+
|Dest|
+----+
| MSY|
| SNA|
| BUR|
| IDA|
| EUG|
| OAK|
| RDM|
| MOD|
| CEC|
| LIH|
| IAH|
| HNL|
| CVG|
| SJC|
| RDD|
| AUS|
| GJT|
| LGB|
| BFL|
| RNO|
+----+
only showing top 20 rows



In [ ]:
print(df.count())

539895


In [ ]:
print(df.collect())

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
print(df.take(3))

[Row(Year='2005', Month='1', DayofMonth='1', DayOfWeek='6', DepTime='1211', CRSDepTime='1216', ArrTime='1451', CRSArrTime='1502', UniqueCarrier='UA', FlightNum='548', TailNum='N341UA', ActualElapsedTime='100', CRSElapsedTime='106', AirTime='81', ArrDelay='-11', DepDelay='-5', Origin='SFO', Dest='SLC', Distance='599', TaxiIn='2', TaxiOut='17', Cancelled='0', CancellationCode=None, Diverted='0', CarrierDelay='0', WeatherDelay='0', NASDelay='0', SecurityDelay='0', LateAircraftDelay='0'), Row(Year='2005', Month='1', DayofMonth='2', DayOfWeek='7', DepTime='1209', CRSDepTime='1216', ArrTime='1447', CRSArrTime='1502', UniqueCarrier='UA', FlightNum='548', TailNum='N398UA', ActualElapsedTime='98', CRSElapsedTime='106', AirTime='79', ArrDelay='-15', DepDelay='-7', Origin='SFO', Dest='SLC', Distance='599', TaxiIn='2', TaxiOut='17', Cancelled='0', CancellationCode=None, Diverted='0', CarrierDelay='0', WeatherDelay='0', NASDelay='0', SecurityDelay='0', LateAircraftDelay='0'), Row(Year='2005', Month

In [ ]:
## filter
spark.sql("""SELECT Origin, Dest, Distance
FROM airlines WHERE distance > 1000
ORDER BY distance DESC""").show(10)

+------+----+--------+
|Origin|Dest|Distance|
+------+----+--------+
|   SFO| BOS|    2704|
|   SFO| BOS|    2704|
|   SFO| BOS|    2704|
|   SFO| BOS|    2704|
|   SFO| BOS|    2704|
|   SFO| BOS|    2704|
|   SFO| BOS|    2704|
|   SFO| BOS|    2704|
|   SFO| BOS|    2704|
|   SFO| BOS|    2704|
+------+----+--------+
only showing top 10 rows



In [ ]:
## aggregation
spark.sql("""SELECT concat(Origin, Dest) as market,
avg(Distance)
FROM airlines 
GROUP BY 1
ORDER BY 1 DESC""").show(10)

+------+-----------------------------+
|market|avg(CAST(Distance AS DOUBLE))|
+------+-----------------------------+
|SFOTWF|                        536.0|
|SFOTUS|                        751.0|
|SFOSTL|                       1736.0|
|SFOSNA|                        372.0|
|SFOSMX|                        216.0|
|SFOSMF|                         86.0|
|SFOSLC|                        599.0|
|SFOSJC|                         30.0|
|SFOSEA|                        679.0|
|SFOSBP|                        191.0|
+------+-----------------------------+
only showing top 10 rows



In [ ]:
## aggregation
spark.sql("""SELECT concat(Origin, Dest) as market,
min(Distance)
FROM airlines 
GROUP BY 1
ORDER BY 1 DESC""").show(10)

+------+-------------+
|market|min(Distance)|
+------+-------------+
|SFOTWF|          536|
|SFOTUS|          751|
|SFOSTL|         1736|
|SFOSNA|          372|
|SFOSMX|          216|
|SFOSMF|           86|
|SFOSLC|          599|
|SFOSJC|           30|
|SFOSEA|          679|
|SFOSBP|          191|
+------+-------------+
only showing top 10 rows



In [ ]:
## Top 10
spark.sql("""SELECT concat(Origin, Dest) as market,
max(Distance)
FROM airlines 
GROUP BY 1
ORDER BY max(Distance) DESC
limit 10""").show(10)

+------+-------------+
|market|max(Distance)|
+------+-------------+
|SFOELP|          993|
|SFODEN|          967|
|SFOCOS|          963|
|SFOBIL|          909|
|SFOABQ|          896|
|SFOSMF|           86|
|SFOASE|          848|
|SFOEGE|          847|
|SFOFCA|          844|
|SFOBZN|          807|
+------+-------------+

